In [1]:
import tensorflow as tf
import numpy as np
import os
import wave

In [16]:
class DBM():
    def __init__(self, layers, unit_type="bin", noise_std = 0.001, model_path='./models/dbm.ckpt'):
        tf.reset_default_graph()
        self.unit_type = unit_type
        self.layers_nums = layers
        self.layers = []
        self.noise_std = noise_std
        self.input = tf.placeholder("bool" if self.unit_type == "bin" else 'float', [None, None], name="input")
        self.model_path = model_path
        self.loaded_model = False
        
        i = 0
        for unit_count in layers:
            self.layers.append({
                "state" : None,
                "b" : tf.Variable(tf.zeros([unit_count]), name="b_"+str(i))
            })
            i+=1
        self.W = []
        
        for i in range(len(layers)-1):
            self.W.append(tf.Variable(tf.random_normal((layers[i], layers[i+1]), mean=0.0, stddev=0.01), name="W_"+str(i)))
        
        self.tf_saver = tf.train.Saver()
        
    def __gaussian_noise_layer(self, x):
        noise = tf.random_normal(shape=tf.shape(x), mean=0.0, stddev=self.noise_std, dtype=tf.float32) 
        return x + noise
    
    def sample_forward(self, start_from=0, steps_num=None):
        if(steps_num == None or steps_num > len(self.layers)):
            steps_num = len(self.layers)-1
        for i in range(start_from, start_from + steps_num):
            mul = tf.matmul(self.layers[i]["state"], self.W[i]) + self.layers[i+1]["b"]
            
            if(self.unit_type == "bin") :
                self.layers[i+1]["state"] = tf.nn.relu(tf.sign(tf.sigmoid(mul) 
                                                               - np.random.rand(tf.shape(mul)[0], tf.shape(mul)[1])))
            else:
                 self.layers[i+1]["state"] = tf.nn.relu(self.__gaussian_noise_layer(mul))
                    
        return self.layers[i+1]["state"]
    
    def sample_back(self, start_from=None, steps_count=None):
        if(steps_count == None):
            steps_count = len(self.layers)-1
            start_from = len(self.layers)-1
        i = start_from
        while(i > start_from - steps_count):
            mul = tf.matmul(self.layers[i]["state"], tf.transpose(self.W[i-1])) + self.layers[i-1]["b"]
            if(self.unit_type == "bin"):
                self.layers[i-1]["state"] = tf.nn.relu(tf.sign(tf.sigmoid(mul) 
                                                               - np.random.rand(tf.shape(mul)[0], tf.shape(mul)[1])))
            else:
                 self.layers[i-1]["state"] = self.__gaussian_noise_layer(mul)
            i -= 1
        return self.layers[i]["state"]
    
    def __getNextBatch(self, batch_size):
        if(self.train_set):
            if(callable(self.train_set)):
                gen = self.train_set(batch_size)
                for result in gen:
                    yield result
                    
    def prepare_train_set(self, batch_size, epochs_count):
        if(callable(self.train_set)):
            return self.train_set(self.files, int(self.layers_nums[0]), batch_size, epochs_count, True)
    
    def train(self, train_set, batch_size, learning_rate, epochs_count, decrease_noise = 0, pcd = 1, depth=None):
        if(depth == None or depth > len(self.layers)-1):
            depth = len(self.layers)
        self.train_set = train_set
        if(self.loaded_model == False):
            self.tf_sess = tf.Session()
            self.tf_sess.run(tf.global_variables_initializer())
        self.train_set = self.prepare_train_set(batch_size, epochs_count*(depth-1))
        print("start training "+str(learning_rate))
        for i in range(depth-1):
            self.noise_std = 0.001
            inp = self.train_set.__next__()
            for j in  range(epochs_count):
                self.noise_std = self.noise_std - decrease_noise*self.noise_std
                self.layers[0]["state"] = self.input
                h0_state = self.sample_forward(0, i+1)
                v0_state = self.layers[i]["state"]
                positive = tf.matmul(tf.transpose(v0_state), h0_state)
                for k in range(pcd):                    
                    v1_state = self.sample_back(i+1, 1)
                    h1_state = self.sample_forward(i, 1)
                    negative = tf.matmul(tf.transpose(v1_state), h1_state)
                    w_update = self.W[i].assign_add(learning_rate*(positive - negative))
                    v_loss = tf.reduce_mean(v0_state - v1_state, 0)
                    vb_update = self.layers[i]["b"].assign_add(learning_rate*(v_loss))
                    hb_update = self.layers[i+1]["b"].assign_add(learning_rate*(tf.reduce_mean(h0_state - h1_state, 0)))
                    w_upd, bv_upd, bh_upd, loss = self.tf_sess.run([w_update, vb_update, hb_update, v_loss], feed_dict={
                        self.input : norm(inp)
                    })
                if(j % 100 == 0):
                    print("epoch: "+str(j)+" layer:"+str(i)+" loss:")
                    print(np.mean(np.power(loss, 2)))
        self.train_set.__next__()
        self.tf_saver.save(self.tf_sess, self.model_path)
        
    def load_model(self, path=None):
        self.tf_sess = tf.Session()
        self.tf_sess.run(tf.global_variables_initializer())
        self.tf_saver.restore(self.tf_sess, path if path != None else self.model_path)
        self.loaded_model = True
        
    def encode(self, data):
        self.layers[0]["state"] = self.input
        res = self.sample_forward()
        with tf.Session() as self.tf_sess:
           # print("WAT")
            self.tf_sess.run(tf.global_variables_initializer())
            self.tf_saver.restore(self.tf_sess, self.model_path)
            return self.tf_sess.run(res, feed_dict={
                self.input : data
            })
    
    def decode(self, data):
        self.layers[-1]["state"] = self.input
        res = self.sample_back()
        with tf.Session() as self.tf_sess:
            self.tf_sess.run(tf.global_variables_initializer())
            self.tf_saver.restore(self.tf_sess, self.model_path)
            return self.tf_sess.run(res, feed_dict={
                self.input : data
            })
        
    def loss(self):
        self.layers[0]["state"] = self.input
        return np.mean(np.pow(self.sample_back(self.sample_forvard(data) - self.input)))
    
    def finetune(self, train_set, batch_size, learning_rate, epochs_count):
        self.train_set = train_set
        if(self.decode_layers !=None):
            self.decode_layers = []
            for i in range(len(layers)-1):
                self.decode_layers.append({
                    "state" : None,
                    "b" : self.layers[i]["b"]
                })
                i+=1

            decode_W.W = []
            for i in range(len(layers)-1):
                self.decode_W.append(tf.transpose(self.W[i]))
                
        mimmaze = tf.train.AdamOptimizer(learning_rate).minimize(self.loss())
        self.train_set = self.prepare_train_set(batch_size, len(self.layers))
        
        with tf.Session() as self.tf_sess:
            self.tf_sess.run(tf.global_variables_initializer())
            self.tf_saver.restore(self.tf_sess, self.model_path)
            for i in range(epochs_count):
                 self.tf_sess.run(mimmaze, feed_dict={
                    self.input : norm(self.train_set.__next__())
                })
                
    def compute_loss(self, data):
        return np.mean(np.pow(self.decode(self.encode(data) - data)))
    
    def regression(self, train_set, batch_size, epoch_count):
        for i in range(len(self.layers_nums)):
            tf.stop_gradient(self.layers[i]["b"])
            if(i < len(self.layers_nums)-1):
                tf.stop_gradient(self.W[i])
        file_labels = tf.placeholder(tf.int64, [None], name="Labels")
        self.train_set = train_set
        if(self.loaded_model == False):
            self.load_model()
        self.train_set = self.prepare_train_set(batch_size, epoch_count*(len(self.layers)), True)
        for j in range(epoch_count):
            self.layers[0]["state"] = self.input
            res = self.sample_forward()
            mul = tf.matmul(res, self.rW) + self.rb
            cross_entropy = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=mul, labels=file_labels))
            train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
            batch = self.train_set.__next__()
            err, step = self.tf_sess.run([cross_entropy, train_step], feed_dict = {
                self.input : norm(batch[0]),
                file_labels : batch[1]
            })
            if(j % 100 == 0):
                print("epoch: "+str(j)+" loss:")
                print(err)
        self.train_set.__next__()
        self.tf_saver.save(self.tf_sess, self.model_path)

    def transform(self, inp, target, k):
        self.layers[0]["state"] = self.input
        encoded = self.sample_forward()
        self.layers[-1]["state"] = encoded + k*tf.transpose(self.rW)[target]
        res = self.sample_back()
        if(self.loaded_model == False):
            self.load_model()
        enc, res = self.tf_sess.run([encoded, res], feed_dict={
            self.input : inp
        })
        print(self.tf_sess.run(tf.transpose(self.rW)[target]))
        return res

In [17]:
def norm(x, get_mu_std=False):
    ar = np.array(x)
    mu = np.mean(ar)
    std = np.std(ar)
    if(get_mu_std):
        return (ar - mu)/std, mu, std
    else:
        return (ar - mu)/std

def restore(x, mu, std):
    return np.array(x)*std + mu

def get_file_frame(file_path, frame_with, frame_count, batch_count, rand=False, get_meta=False, get_label=False):
    if(type(file_path ) == str):
        wave_read = [wave.open(file_path, "rb")]
    else:
        wave_read = []
        for fp in file_path:
            wave_read.append (wave.open(fp, "rb"))
    if(get_meta):
        meta = {
            "nframes" : wave_read[0].getnframes(),
            "nchannels" : wave_read[0].getnchannels(),
            "sampwidth" : wave_read[0].getsampwidth(),
            "framerate" : wave_read[0].getframerate()
        }
    for k in range(batch_count):
        out = []
        labels = []
        for i in range(frame_count):
            label = np.random.randint(len(wave_read)) - 1
            chosen_file = wave_read[label]
            if rand:
                chosen_file.setpos(np.random.randint(chosen_file.getnframes() - frame_with))           
            if(get_label == True):
                labels.append(label)
            out.append(np.fromstring(chosen_file.readframes(frame_with), np.uint16))
        yield [out, labels] if(get_label == True) else out
    for fr in wave_read:
        fr.close()
    yield meta if get_meta else None
    
def write_wave(array, path, meta, mu=None, std=None):
    unrolled = array.ravel()
    writer = wave.open(path, "wb")
    writer.setnframes(meta["nframes"])
    unrolled = restore(unrolled, mu, std)
    bytes_arr = np.rint(unrolled).astype(np.uint16).tobytes()
    writer.setnchannels(meta["nchannels"])
    writer.setsampwidth(meta["sampwidth"])
    writer.setframerate(meta["framerate"])
    writer.writeframes(bytes_arr)
    writer.close()

def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

In [ ]:
rbm = DBM([500, 300, 200, 100], "gauss")
rbm.files= ["01.wav", "03.wav", "04.wav", "05.wav", "06.wav", "07.wav", "11.wav" ,"13.wav", "14.wav"]
rbm.train(get_file_frame, 500, 0.0000015, 700, 0.002, 5)

start training 1.5e-06
epoch: 0 layer:0 loss:
0.00274478
epoch: 100 layer:0 loss:
0.000312765
epoch: 200 layer:0 loss:
9.32763e-05
epoch: 300 layer:0 loss:
6.26744e-05
epoch: 400 layer:0 loss:
4.18432e-05
epoch: 500 layer:0 loss:
2.7766e-05
epoch: 600 layer:0 loss:
1.87814e-05
epoch: 0 layer:1 loss:
0.297422
epoch: 100 layer:1 loss:
8.00387e-05
epoch: 200 layer:1 loss:
5.87564e-05
epoch: 300 layer:1 loss:
4.25602e-05
epoch: 400 layer:1 loss:
3.51944e-05
epoch: 500 layer:1 loss:
2.75689e-05
epoch: 600 layer:1 loss:
2.22493e-05
epoch: 0 layer:2 loss:
0.522653
epoch: 100 layer:2 loss:
6.86326e-05
epoch: 200 layer:2 loss:
4.10274e-05
epoch: 300 layer:2 loss:
3.07687e-05
epoch: 400 layer:2 loss:
2.77465e-05
epoch: 500 layer:2 loss:
2.55883e-05
epoch: 600 layer:2 loss:
2.33229e-05


In [5]:
rbm = DBM([500, 300, 200, 100], "gauss")
rbm.load_model()
gen = get_file_frame("01.wav", 500, 500, 1, False, True)
foo = gen.__next__()  
foo, mu, std = norm(foo, True)
res = rbm.encode(foo) 
meta = gen.__next__()

In [6]:
decoded = rbm.decode(res)

In [ ]:
write_wave(decoded, "ress.wav", meta, mu, std) 

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

In [134]:
class VariationalAutoencoder(object):
    """ Variation Autoencoder (VAE) with an sklearn-like interface implemented using TensorFlow.
    
    This implementation uses probabilistic encoders and decoders using Gaussian 
    distributions and  realized by multi-layer perceptrons. The VAE can be learned
    end-to-end.
    
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct= tf.nn.relu, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.global_variables_initializer()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        network_weights = self._initialize_weights(**self.network_architecture)

        # Use recognition network to determine mean and 
        # (log) variance of Gaussian distribution in latent
        # space
        self.z_mean, self.z_log_sigma_sq = \
            self._recognition_network(network_weights["weights_recog"], 
                                      network_weights["biases_recog"])

        # Draw one sample z from Gaussian distribution
        n_z = self.network_architecture["n_z"]
        eps = tf.random_normal((self.batch_size, n_z), 0, 1, 
                               dtype=tf.float32)
        # z = mu + sigma*epsilon
        self.z = tf.add(self.z_mean, 
                        tf.multiply(tf.sqrt(tf.exp(self.z_log_sigma_sq)), eps))

        # Use generator to determine mean of
        # Bernoulli distribution of reconstructed input
        self.x_reconstr_mean = \
            self._generator_network(network_weights["weights_gener"],
                                    network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  n_hidden_gener_2, 
                            n_input, n_z):
        all_weights = dict()
        all_weights['weights_recog'] = {
            'h1': tf.Variable(xavier_init(n_input, n_hidden_recog_1)),
            'h2': tf.Variable(xavier_init(n_hidden_recog_1, n_hidden_recog_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_recog_2, n_z)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_recog_2, n_z))}
        all_weights['biases_recog'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))}
        all_weights['weights_gener'] = {
            'h1': tf.Variable(xavier_init(n_z, n_hidden_gener_1)),
            'h2': tf.Variable(xavier_init(n_hidden_gener_1, n_hidden_gener_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_gener_2, n_input)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_gener_2, n_input))}
        all_weights['biases_gener'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_gener_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_input], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_input], dtype=tf.float32))}
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network), which
        # maps inputs onto a normal distribution in latent space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        z_mean = tf.add(tf.matmul(layer_2, weights['out_mean']),
                        biases['out_mean'])
        z_log_sigma_sq = \
            tf.add(tf.matmul(layer_2, weights['out_log_sigma']), 
                   biases['out_log_sigma'])
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self, weights, biases):
        # Generate probabilistic decoder (decoder network), which
        # maps points in latent space onto a Bernoulli distribution in data space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.z, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        x_reconstr_mean = (tf.add(tf.matmul(layer_2, weights['out_mean']), 
                                 biases['out_mean']))
        return x_reconstr_mean
            
    def _create_loss_optimizer(self):
        # The loss is composed of two terms:
        # 1.) The reconstruction loss (the negative log probability
        #     of the input under the reconstructed Bernoulli distribution 
        #     induced by the decoder in the data space).
        #     This can be interpreted as the number of "nats" required
        #     for reconstructing the input when the activation in latent
        #     is given.
        # Adding 1e-10 to avoid evaluation of log(0.0)
        reconstr_loss = tf.reduce_mean(tf.pow(self.x - self.x_reconstr_mean, 2))
        # 2.) The latent loss, which is defined as the Kullback Leibler divergence 
        ##    between the distribution in latent space induced by the encoder on 
        #     the data and some prior. This acts as a kind of regularizer.
        #     This can be interpreted as the number of "nats" required
        #     for transmitting the the latent space distribution given
        #     the prior.
        latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
                                           - tf.square(self.z_mean) 
                                           - tf.exp(self.z_log_sigma_sq), 1)
        self.cost = tf.reduce_mean(reconstr_loss + latent_loss)   # average over batch

        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
        opt, cost = self.sess.run((self.optimizer, self.cost), 
                                  feed_dict={self.x: X})
        return cost
    
    def transform(self, X):
        """Transform data by mapping it into the latent space."""
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        return self.sess.run(self.z_mean, feed_dict={self.x: X})
    
    def generate(self, z_mu=None):
        """ Generate data by sampling from latent space.
        
        If z_mu is not None, data for this point in latent space is
        generated. Otherwise, z_mu is drawn from prior in latent 
        space.        
        """
        if z_mu is None:
            z_mu = np.random.normal(size=self.network_architecture["n_z"])
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.z: z_mu})
    
    def reconstruct(self, X):
        """ Use VAE to reconstruct given data. """
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.x: X})

In [135]:
np.array([2,0,3]) != 0

array([ True, False,  True], dtype=bool)

In [148]:
network_architecture = \
    dict(n_hidden_recog_1=500, # 1st layer encoder neurons
         n_hidden_recog_2=300, # 2nd layer encoder neurons
         n_hidden_gener_1=300, # 1st layer decoder neurons
         n_hidden_gener_2=500, # 2nd layer decoder neurons
         n_input=500, # MNIST data input (img shape: 28*28)
         n_z=100)

In [151]:
gen = get_file_frame(["01.wav"], 500, 500, 10000, rand=True, get_meta=False, get_label=False);
def train(network_architecture, learning_rate=0.0005,
          batch_size=500, training_epochs=10000, display_step=100):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples / batch_size)
        # Loop over all batches
        
        batch_xs= norm(gen.__next__());
        # Fit training using batch data
        cost = vae.partial_fit(batch_xs)
        # Compute average loss
        avg_cost += cost / batch_size

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), 
                  "cost=", "{:.9f}".format(cost))
    return vae

In [152]:
vae = train(network_architecture)

Epoch: 0001 cost= 32.197929382
Epoch: 0101 cost= 1.019004464
Epoch: 0201 cost= 1.008387685
Epoch: 0301 cost= 1.005912304
Epoch: 0401 cost= 1.002282619
Epoch: 0501 cost= 1.001438498
Epoch: 0601 cost= 1.001357436
Epoch: 0701 cost= 1.001116037
Epoch: 0801 cost= 1.000467181
Epoch: 0901 cost= 1.000377774
Epoch: 1001 cost= 1.001035333
Epoch: 1101 cost= 1.000309229
Epoch: 1201 cost= 1.000334382
Epoch: 1301 cost= 1.000373721
Epoch: 1401 cost= 1.000386000
Epoch: 1501 cost= 1.000252485
Epoch: 1601 cost= 1.000172615
Epoch: 1701 cost= 1.000199556
Epoch: 1801 cost= 1.000069857
Epoch: 1901 cost= 1.000148892
Epoch: 2001 cost= 1.000207424
Epoch: 2101 cost= 1.000118613
Epoch: 2201 cost= 1.000210524
Epoch: 2301 cost= 1.000092030
Epoch: 2401 cost= 1.000321150
Epoch: 2501 cost= 1.000034928
Epoch: 2601 cost= 1.000141144
Epoch: 2701 cost= 0.999970675
Epoch: 2801 cost= 0.999992907
Epoch: 2901 cost= 1.000103831
Epoch: 3001 cost= 1.000164866
Epoch: 3101 cost= 1.000364065
Epoch: 3201 cost= 1.000015855
Epoch: 33

In [154]:
gen = get_file_frame("01.wav", 500, 500, 1, False, True)
foo = gen.__next__()  
foo, mu, std = norm(foo, True)
res = vae.reconstruct(foo)
meta = gen.__next__()
write_wave(res, "ress.wav", meta, mu, std) 

In [94]:
np.shape(foo)

(500, 500)

In [155]:
res

array([[ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899],
       [ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899],
       [ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899],
       ..., 
       [ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899],
       [ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899],
       [ 0.00384825,  0.00057701,  0.00421493, ...,  0.00760673,
         0.00468805,  0.00405899]], dtype=float32)